<a href="https://colab.research.google.com/github/mexemt/datascience/blob/master/Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import itertools
import numpy as np
from textblob import TextBlob
import re
import sklearn
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from nltk.probability import FreqDist
from nltk.stem.porter import PorterStemmer

In [13]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['source']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Datenvorbereitung

In [14]:
url = 'https://raw.githubusercontent.com/mexemt/datascience/master/dataset/'

filepath_dict = {'yelp':   url+'yelp_labelled.txt',
                 'amazon': url+'amazon_cells_labelled.txt',
                 'imdb':   url+'imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[0])

sentence    Wow... Loved this place.
label                              1
source                          yelp
Name: 0, dtype: object


Algorithmen

In [0]:
classifierLogistic = LogisticRegression()
classifierSGD = SGDClassifier(loss='squared_hinge', alpha=0.0001, tol=0.00001, random_state=0, eta0=0.0001)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
clf = svm.SVC()
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(4, 12), max_iter=1000, random_state=0)





def Logistic(X_train, X_test, y_train, portal, y_test):
    classifierLogistic.fit(X_train, y_train)
    y_pred = classifierLogistic.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    res_cv = cross_validate(classifierLogistic, X_train, y_train, cv=5)
    print(res_cv)
    print('Logistic Regression Accuracy for {} data: {:.4f}'.format(portal, score))

def SGD(X_train, X_test, y_train, portal, y_test):
    classifierSGD.fit(X_train, y_train)
    y_pred = classifierSGD.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    res_cv = cross_validate(classifierSGD, X_train, y_train, cv=5)
    print(res_cv)
    print('classifierSGD Accuracy for {} data: {:.4f}'.format(portal, score))

def RF(X_train, X_test, y_train, portal, y_test):
    classifier.fit(X_train, y_train) 
    y_pred = classifier.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    res_cv = cross_validate(classifier, X_train, y_train, cv=5)
    print(res_cv)
    print('Random Forest Accuracy for {} data: {:.4f}'.format(portal, score))

def SVC(X_train, X_test, y_train, portal, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    res_cv = cross_validate(clf, X_train, y_train, cv=5)
    print(res_cv)
    print('Support Vector Machine Accuracy for {} data: {:.4f}'.format(portal, score))


def MLP(X_train, X_test, y_train, portal, y_test):
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    res_cv = cross_validate(mlp, X_train, y_train, cv=5)
    print(res_cv)
    print('MLP Accuracy for {} data: {:.4f}'.format(portal, score))



Anwendung

In [0]:
def ML_Classifiers(portal):
  scoring = ['accuracy', 'recall', 'precision', 'f1']

  df_source = df[df['source'] == portal]
  sentences = df_source['sentence'].values
  y = df_source['label'].values

  sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.35, random_state=0)

  vectorizer = TfidfVectorizer(stop_words='english')
  vec = vectorizer.fit(sentences_train)
  vec

  X_train = vectorizer.transform(sentences_train)
  X_test  = vectorizer.transform(sentences_test)

  vec = vectorizer.fit(sentences_train)
  bag_of_words = vec.transform(sentences_train)
  sum_words = bag_of_words.sum(axis=0)
  words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
  words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

  print(words_freq)

  Logistic(X_train, X_test, y_train, portal, y_test)
  SGD(X_train, X_test, y_train, portal, y_test)
  RF(X_train, X_test, y_train, portal, y_test)
  SVC(X_train, X_test, y_train, portal, y_test)
  MLP(X_train, X_test, y_train, portal, y_test)



Auswertung

In [17]:
ML_Classifiers('amazon')

[('phone', 30.557693554938695), ('great', 26.213063112323276), ('good', 17.536202016697708), ('product', 15.266921684711663), ('headset', 12.05371195812412), ('works', 11.827071772436346), ('price', 10.486372224028129), ('quality', 10.29096533760763), ('use', 9.834236227378176), ('excellent', 9.549870225806563), ('recommend', 9.432950156747259), ('don', 9.218135017422782), ('work', 9.210072874032656), ('battery', 8.6783502860824), ('sound', 8.436638171582608), ('service', 7.682235354611475), ('really', 7.245855606716428), ('time', 6.947568297645257), ('ear', 6.738808982771774), ('does', 6.570552879693774), ('happy', 6.329331444086515), ('bad', 6.285122231740049), ('like', 6.212198145102423), ('ve', 6.20210996210544), ('just', 5.989904848786346), ('best', 5.912106603842735), ('love', 5.825315128766977), ('doesn', 5.7376683873750345), ('money', 5.736835006199774), ('new', 5.5682090910285496), ('waste', 5.562214688398266), ('bluetooth', 5.54559331811253), ('easy', 5.51818127759092), ('wor

In [18]:
ML_Classifiers('yelp')

[('food', 22.252703297361105), ('good', 22.237941202074012), ('place', 21.677474804527108), ('great', 17.28944651077793), ('service', 17.048194619095796), ('like', 9.482207351834646), ('really', 8.992235799994765), ('won', 8.121927966172391), ('best', 8.096594185519702), ('time', 7.92649821562906), ('amazing', 7.8926143204551975), ('just', 7.828501530021438), ('delicious', 7.566988516864459), ('ve', 6.601029181000219), ('definitely', 6.429100922377474), ('friendly', 6.270601677774444), ('nice', 6.136190636040704), ('don', 5.880134067292856), ('experience', 5.847032822725482), ('love', 5.835319557185967), ('restaurant', 5.774048216114429), ('disappointed', 5.610392534118063), ('wait', 5.454372331456021), ('going', 5.4347225343190715), ('did', 5.369416092623098), ('recommend', 5.344783385428725), ('worst', 5.304665442812722), ('probably', 5.199528374853972), ('better', 5.010738086563396), ('staff', 4.939623279606481), ('eat', 4.915503233676281), ('bad', 4.905307328847028), ('came', 4.896

In [19]:
ML_Classifiers('imdb')

[('movie', 21.95239643741207), ('film', 17.70819647744573), ('bad', 12.510444716944725), ('just', 8.666946194603085), ('good', 8.233538377199993), ('great', 6.867969964597559), ('really', 6.716099063274612), ('acting', 6.081792619278555), ('like', 5.669003209649147), ('10', 5.417474408742768), ('time', 5.342089644754686), ('plot', 5.098200601274956), ('characters', 5.0099850236861485), ('stupid', 4.433236913689966), ('don', 4.363957031149667), ('real', 4.331472139467905), ('look', 4.32404030792924), ('actors', 4.046814118453828), ('watch', 4.039604896060723), ('story', 3.8970149684784663), ('love', 3.868633126987706), ('script', 3.8330539446355423), ('liked', 3.830595693762667), ('way', 3.7872691595044854), ('seen', 3.7455054010521778), ('awful', 3.736547325046103), ('make', 3.6789748274633243), ('enjoyed', 3.631683965623165), ('think', 3.6306723949188324), ('wonderful', 3.4829643330206004), ('films', 3.418069613064815), ('music', 3.408790210224361), ('recommend', 3.3651994367775995), 